<a href="https://colab.research.google.com/github/vinayshanbhag/azure_custom_vision/blob/main/Azure_Custom_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product search using Azure Custom Vision Service

Azure Custom Vision is an image recognition service that lets you build, deploy, and improve your own image identifier models. An image identifier applies labels to images, according to their detected visual characteristics. Each label represents a classifications or objects. Unlike the Computer Vision service, Custom Vision allows you to specify your own labels and train custom models to detect them.

The Custom Vision service uses a machine learning algorithm to analyze images. You, the developer, submit groups of images that have and don't have the characteristics in question. You label the images yourself at the time of submission. Then the algorithm trains to this data and calculates its own accuracy by testing itself on those same images. Once you've trained the algorithm, you can test, retrain, and eventually use it in your image recognition app to classify images. You can also export the model itself for offline use.

Multi-class (not multi-label) classifier is explored here. Each tag (product type) must have at least 5 images in the training set. 

# Installation

```pip install azure-cognitiveservices-vision-customvision```

In [2]:
!pip install azure-cognitiveservices-vision-customvision

In [81]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from azure.cognitiveservices.vision.customvision.training.models import Classifier
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

import pandas as pd
from IPython.display import display, HTML

# Azure Subscription

Azure subscription - [Create one for free](https://azure.microsoft.com/free/cognitive-services/)

Update keys and resource ids below. These are found under Settings at [customvision.ai](https://www.customvision.ai/projects)

In [29]:
# Replace with valid values
ENDPOINT = "your_endpoint"
training_key = "your_training_key"
prediction_key = "your_prediction_key"
prediction_resource_id = "your_prediction_resource_id"

In [30]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

Create a Custom Vision project

In [53]:
publish_iteration_name = "makemodelclassification"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Create a new project
print ("Creating project...")
project_name = uuid.uuid4()
project = trainer.create_project(project_name,classification_type=Classifier.multiclass)

Creating project...


Download product images. Select products that have atleast 5 images.
Training will fail if a tag has less than 5 images associated with it.
Free tier is limited to 50 tags (unique products)


In [32]:
!git clone https://github.com/vinayshanbhag/images.git

fatal: destination path 'images' already exists and is not an empty directory.


In [54]:
df = pd.read_csv("/content/images/products.csv", header=None)
df_temp = df[[0,5]].copy()
df_temp.columns = ['path','tag']
# Pick only products with 5 or more images per tag
#df_temp = df_temp[df_temp.tag.isin(df_temp.tag.value_counts().loc[lambda x: x>=5].index)].copy()
allowed_tags = ['Apple MacBook Pro 14 2021', 'Sonos One', 'Amazon Echo Show 10',
       'Google Nest Audio', 'Sony PS4 Console', 
       'Apple iPad Pro 11', 'Apple AirPods Max', 'Sonos Beam Gen2',
       'HP ProBook', 'Sonos Playbar', 'Google Nest Wifi', 'Sonos Five',
       'Apple Magic Trackpad', 'Oculus Quest2', 'Apple MacBook Air 2020',
       'Apple HomePod Mini', 'Ring Video Doorbell', 'HP Deskjet 3755',
       'Sonos Move', 'Google Wifi', 'Sonos Play1', 'HP EliteBook',
       'Sonos Roam', 'Apple AirPods Pro', 'Apple TV 4K',
       'Google Nest Mini', 'Linksys Hydra Pro Axe6600',
       'HP Spectre X360', 'Linksys Velop Ac2200', 'iRobot Roomba I3 Evo',
       'Apple AirPods 3', 'Amazon Echo Show', 'Apple iPad Mini',
       'Google Nest Thermostat E', 'Apple Watch SE', 'Apple iPad Air',
       'Apple Watch Series 7', 'Microsoft Surface Pro',
       'Nintendo Switch Console', 'Sonos Sub Gen3',
       'Apple Magic Keyboard Touch ID Numeric Keypad',
       'Amazon Echo Dot 4', 'Roku Ultra 4K']
df_temp = df_temp[df_temp.tag.isin(allowed_tags)].copy()
len(df_temp.tag.unique())

43

Prepare images and tags for upload

In [55]:
#update local file paths
df_temp['path'] = df_temp.path.apply(lambda x:"/content/images/preprocessed/"+x.split("/")[-1])

In [56]:
# create tags
str_tags = df_temp.tag.unique()
tag_dict = dict() #{str_tag: Tag}
for tag in str_tags:
  t = trainer.create_tag(project.id, tag)
  tag_dict[tag]=t

In [57]:
image_list = []
tag_list = []
for row in df_temp.values:
  with open(row[0],'rb') as image_contents:
    image_list.append(ImageFileCreateEntry(name=row[0].split("/")[-1], contents=image_contents.read(), tag_ids=[tag_dict[row[1]].id]))
    tag_list.append(row[1])

In [58]:
n= len(image_list)
n

238

Atmost 64 images/tags can be uploaded at a time. Split into batches of 64 and upload

In [59]:
batch_size = 64
for i in range(0,n,batch_size):
  if (i+batch_size)<n:
    upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list[i:i+batch_size]))
    if not upload_result.is_batch_successful:
      print("Image batch upload failed.")
      for idx,image in enumerate(upload_result.images):
        print("Image status: ", image.status, tag_list[idx])
      exit(-1)
  else:
    upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list[i:]))
    if not upload_result.is_batch_successful:
      print("Image batch upload failed.")
      for idx, image in enumerate(upload_result.images):
        print("Image status: ", image.status, tag_list[idx])
      exit(-1)

Initiate training after fixing any upload errors

In [60]:
print ("Training...")
iteration = trainer.train_project(project.id)
while iteration.status != "Completed":
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

0it [00:00, ?it/s]

Publish iteration after training is complete

In [65]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


Open the project just created from [customvision.ai](https://www.customvision.ai/projects). Prediction enpoints an keys are under Performance | Prediction URL.

The instructions vary depending on how an image is provided - an image url vs an image file.

Pass a test image url of a product that is in the training set. This image should not be from the training set. 

In [96]:
import requests
endpoint_url="your_endpoint_url_here"
headers={"Prediction-Key":"<your_prediction_key_here",
         "Content-Type":"application/json"}
# Replace with any image of a product (from the training set) that isn't in the set of images used for training
test_image_url = "https://raw.githubusercontent.com/vinayshanbhag/test_images/main/apple_macbook_air_2020_1.jpg" 
body = {"Url": test_image_url}
display(HTML(f"<p>Test Image:</p><img src='{test_image_url}' width='150'/>"))

Call Rest API endpoint and display predictions and probability scores

In [99]:
resp = requests.post(endpoint_url,json=body, headers=headers).json()
print("Predictions:")
[(i['tagName'],i['probability']) for i in resp['predictions'][:5]]

Predictions:


[('HP Spectre X360', 0.7564705),
 ('HP EliteBook', 0.06620501),
 ('Apple MacBook Air 2020', 0.05980634),
 ('HP ProBook', 0.034518816),
 ('Sony PS4 Console', 0.016161513)]